# <font color='black'> Методы анализа неоднородных данных и паттерн-анализ. Практическое занятие 4 </font>
## <font color='black'> Модель Differences-in-Differences (DiD) </font>

Давайте рассмотрим модель DiD на примере исследований Eissa, N. и Liebman, J.: авторы изучали эффект EITC (налогового вычета на трудовой доход) на занятость. Мы рассмотрим, как расширение EITC для женщин с детьми в 1993 г. повлияло на занятость женщин.

Краткое описание данных:

* year - год налогообложения
* work - дамми-переменная, показывающая факт занятости (1 - женщина, имеющая работу, 0 - в противном случае)
* age - возраст женщины
* ed - образование женщины (в количестве лет образования)
* children - количество детей у женщины
* nonwhite - дамми-переменная для расовой принадлежности
* unearn - non-labor income (разность семейного дохода и индивидуального дохода женщины)

In [ ]:
import pandas as pd
import statsmodels.formula.api as statf
import matplotlib.pyplot as mpl
%matplotlib inline

In [ ]:
lab5 = pd.read_stata("DiDdata.dta")
lab5 = lab5.dropna()
lab5.head()

Для начала создадим дамми-переменные для обозначения группы воздействия и периода.

In [ ]:
lab5['treated'] = lab5['children'].apply(lambda x: 1 if x > 0 else 0)
lab5['after'] = lab5['year'].apply(lambda x: 1 if x > 1993 else 0)

Проиллюстрируем на графике, как с течением времени изменялись средние значения занятости женщин в группе воздействия и контрольной группе. Этот график позволит понять, соблюдается ли допущение о параллельности трендов.

In [ ]:
fig, axes = mpl.subplots()
lab5.groupby(['year', 'treated']).mean()['work'].unstack().plot(ax = axes)
axes.set_ylabel("Labor Participation")
mpl.axvline(x=1994, color ='r', linestyle = '--')

Рассчитаем DiD-Оценку практически вручную. Для этого нам нужно  

1) посчитать разницу между средними значениями занятости в группе воздействия в период после и период до воздействия

2) посчитать соответствующую разницу для контрольной группы

3) и наконец-то, посчитать DiD (разность упомянутых выше разностей)

In [ ]:
lab5.groupby(['after', 'treated']).mean()['work']

dif_treated = lab5.groupby(['after', 'treated']).mean()['work'].iloc[3] - lab5.groupby(['after', 'treated']).mean()['work'].iloc[1]
dif_nontreated = lab5.groupby(['after', 'treated']).mean()['work'].iloc[2] - lab5.groupby(['after', 'treated']).mean()['work'].iloc[0]

did = dif_treated - dif_nontreated
did

Теперь оценим соответствующую модель DiD. Видим, что оценка коэффициент при переменной взаимодействия совпадает с оценкой DiD, которую мы получили выше.

In [ ]:
did_1 = statf.ols('work ~ treated*after', data=lab5).fit()
print(did_1.summary())

Добавим контрольные переменные и посмотрим, изменился ли значимым образом эффект расширения EITC.

In [ ]:
did_2 = statf.ols('work ~ treated*after + age + ed + nonwhite + unearn', data=lab5).fit()
print(did_2.summary())